# About this notebook


In this notebook you see how you can create with [Gradio](https://gradio.app/) an interactive multimodel chatbot.


# Imports

In [4]:
import gradio as gr
from transformers import pipeline, BitsAndBytesConfig
import torch

# Load model with quantization and launch Gradio

In [5]:
# Model pipeline with quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

# Model to be used
model_id = "llava-hf/llava-1.5-7b-hf"
pipe = pipeline("image-to-text", model=model_id, model_kwargs={"quantization_config": quantization_config})

# Function to update the conversation as requested by Gradio
def update_conversation(new_message, history, image=None):
    if image is None:
            return "Please upload an image, since this is an image to text model."

    prompt = "USER: <image>\n"

    for user_message, assistant_response in history:
        prompt += f"{user_message}ASSISTANT: {assistant_response}USER: "

    prompt += new_message + "ASSISTANT: "

    outputs = pipe(image, prompt=prompt, generate_kwargs={"max_new_tokens": 200, "do_sample": True, "temperature": 0.7})[0]['generated_text']


    return outputs[len(prompt)-5:]



with gr.Blocks() as demo:

    image = gr.Image(type='pil', interactive=True)

    chat = gr.ChatInterface(
        update_conversation,
        additional_inputs=[image]
    )


# Launch the application
demo.launch()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/70.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://09ee5a702c469ac6c2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
